# Lab Recap

Can we retrieve 3 random facts from the [Internet Chuck Norris Database](http://www.icndb.com/api/) and display them as formatted HTML?

In [12]:
import requests
import json
from pyhtml import p, div
from IPython.display import HTML

result = json.loads(requests.get('http://api.icndb.com/jokes/random/3').content)

result

jokes = []

for entry in result['value']:
    jokes.append(p(entry['joke']))

HTML(str(div(jokes[0], jokes[1], jokes[2])))

## Cryptographic Hashing

Key terms:
* Hashing: Transforming data of any size into a fixed-size representation (a hash). Typically:
  * If the input data is changed even slightly, the hash is completely different.
  * If you have the hash, you can't determine the exact input that was used, but you may be able to determine a *possible* input.
* Cryptographic Hashing: Hashing where it's computationally difficult to determine a possible input.

Can we write a simple hash function?

In [13]:
def simple_hash(text):
    result = 0
    for c in text:
        result += ord(c)
    result = result % 256
    return result

In [19]:
simple_hash("This is a very important message. We don't want information to be lost in transmission.")

223

In [26]:
simple_hash('ze')

223

Can we create a cryptographic hash for this string?

In [37]:
important_message = "The exam is on the 12th of May"

import hashlib

hashlib.sha256(important_message.encode()).hexdigest()

'90823e6ea7afcabb93b9d58a9fece349bc002aa880e374c2d0e43187dff37633'

## Digital Signing

By combining hashing and asymmetric encryption, we have a way of "signing" digital documents.

Can we digitally sign the following string?

In [41]:
verified_message = "Don't forget to double-check your answers in an exam. It's really important. Also, go to the toilet before the exam."

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.hashes import SHA256

private_key = rsa.generate_private_key(65537, 2048)


private_key.sign(verified_message.encode(), padding.PKCS1v15(), SHA256())

b'\xb0^\xa5\x8b84W{1\xc0u\xe5\xe7\x02\xc2\xa9x[\x11A\xd1h\tO1W\xfc\xb6,j{\x0fF\xe4@w\xcc\xff\x0f\x18\xd0\xda\x17\xa5w\xac\xf9\xaf"4Z/\xd7[\xfe\x8d\xbdS,\xc0\xd2T\xb9\xc9\x9f\xb6\xd23\xe2\xb5\xd7T]\xe1O\xc3\xa6}2#3\xe2\xee\x01\xa8\x87A\x11S\x97&=\xf15\x9b\xde\xf0\xd6\xbbZ4\xad\xbd\x8a\xee\xe5\x17N*bZ\xf1 \x8e]~1\xb1\x92\xc6\xe1\xfb<\xa2\xcf\xc6\xd9\x1c\xb1\xdbG\x1b\xd4@\x1fAi{\x8c\xbb\x94w \xb1^.\xc9\xb1C\x04\xfa\x81\x8a\x95\xbd\xc3\x05G\xe1\xf9\xaf\xbd\xc3^\xefJy\xa6\\\x85\t\x97r\xa58\x88O7\xc8h\x7f\x1d\x1e\xe5}\xdc\x06\xf6\x89D\xb8\x99m\xe7^\x9cs/\xcfl\xa2\x078\xcd\xa0.\xae\x13\\\xabV\x01\xf1\x97\x1d\xf4#\x1d\xa4\xfb\x7fHW\x92`\xc5\x9a_i\xc8\xbe\xd2\xc9SQQY\x1a1\xce\xb9g$-\xca\r\xa5\x14\xcf/\xe0\xf8"\xe6\\c'

# Searching

Search algorithms are a well studied class of algorithms as many problems can be expressed as search problems.

As an example, consider this "map" of Romania

<img src="https://d3i71xaburhd42.cloudfront.net/437af7588c6a36fd55c410b7f92b7f47ef57653b/5-Figure3.2-1.png" alt="Romania" />

In [47]:
75  + 71 + 151 + 99 + 211

607

In [48]:
140 + 99 + 211

450

In [50]:
140 + 80 + 97 + 101

418

### Depth-first search

In depth-first search, we explore as far as possible along each path before going back and finding another one.

<img src="https://d18l82el6cdm1i.cloudfront.net/uploads/mf7THWHAbL-mazegif.gif" />
<small>Animation from here: https://brilliant.org/wiki/depth-first-search-dfs/</small>

This is the map of Romania as a dictionary of dictionaries:

In [43]:
romania = {
    'Arad': { 'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118 },
    'Zerind': { 'Arad': 75, 'Oradea': 71 },
    'Sibiu': { 'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80 },
    'Timisoara': { 'Arad': 118, 'Lugoj': 111 },
    'Oradea': { 'Zerind': 71, 'Sibiu': 151 },
    'Lugoj': { 'Timisoara': 111, 'Mehadia': 70 },
    'Fagaras': { 'Sibiu': 99, 'Bucharest': 211 },
    'Rimnicu Vilcea': { 'Sibiu': 80, 'Pitesti': 97, 'Craiova': 146 },
    'Mehadia': { 'Lugoj': 70, 'Dobreta': 75 },
    'Bucharest': { 'Fagaras': 211, 'Pitesti': 101, 'Urziceni': 85, 'Giurglu': 90 },
    'Pitesti': { 'Rimnicu Vilcea': 97, 'Bucharest': 101, 'Craiova': 138 },
    'Craiova': { 'Rimnicu Vilcea': 146, 'Pitesti': 138, 'Dobreta': 120 },
    'Dobreta': { 'Mehadia': 75, 'Craiova': 120 },
    'Urziceni': { 'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142 },
    'Giurglu': { 'Bucharest': 90 },
    'Hirsova': { 'Urziceni': 98, 'Eforie': 86 },
    'Vaslui': { 'Urziceni': 142, 'Lasi': 92 },
    'Eforie': { 'Hirsova': 86 },
    'Lasi': { 'Vaslui': 92, 'Neamt': 87 },
    'Neamt': { 'Lasi': 87 }
}

Find a path from Arad to Bucharest using depth first search

In [46]:
def depth_first(path_so_far):
    current_city = path_so_far[-1]
    if current_city == 'Bucharest':
        return path_so_far
    else:
        for neighbour in romania[current_city]:
            if neighbour not in path_so_far:
                path_found = depth_first(path_so_far + [neighbour])
                if path_found:
                    return path_found

depth_first(['Arad'])

['Arad', 'Zerind', 'Oradea', 'Sibiu', 'Fagaras', 'Bucharest']

### Breadth-first search

In breadth-first search, we explore by moving outward from the start along all possible paths.

Find a path from Arad to Bucharest using breadth-first search

In [49]:
def breadth_first():
    paths_so_far = []
    current_path = ["Arad"]
    while current_path[-1] != 'Bucharest':
        current_city = current_path[-1]
        for neighbour in romania[current_city]:
            paths_so_far.append(current_path + [neighbour])
        current_path = paths_so_far.pop(0)
    return current_path

breadth_first()

['Arad', 'Sibiu', 'Fagaras', 'Bucharest']

#### Lab 4 challenge

Write a function, `nearby(city)`, that returns a list containing all the cities within a 300 mile trip of the the given city, assuming you can only travel along the roads given. 

In [59]:
def nearby(city):
    paths_so_far = []
    current_path = ([city], 0)
    explored_cities = set()
    while current_path != None:
        current_city = current_path[0][-1]
        explored_cities.update([current_city])
        for neighbour in romania[current_city]:
            distance = current_path[1] + romania[current_city][neighbour]
            if distance < 300:
                paths_so_far.append((current_path[0] + [neighbour], distance))
        if len(paths_so_far) > 0:
            current_path = paths_so_far.pop(0)
        else:
            current_path = None
    return explored_cities

nearby('Arad')

{'Arad',
 'Fagaras',
 'Lugoj',
 'Mehadia',
 'Oradea',
 'Rimnicu Vilcea',
 'Sibiu',
 'Timisoara',
 'Zerind'}

### Uniform cost search
In uniform-cost search, we explore by moving outward from the start, but exploring along the shortest path found so far first.

Find the shortest path from Arad to Bucharest using uniform-cost search.

In [62]:
import heapq

def uniform_cost():
    paths_so_far = []
    current_path = ["Arad"]
    distance_travelled = 0
    while current_path[-1] != 'Bucharest':
        current_city = current_path[-1]
        for neighbour in romania[current_city]:
            new_distance = distance_travelled + romania[current_city][neighbour]
            path_entry = current_path + [neighbour]
            heapq.heappush(paths_so_far, (new_distance, path_entry))
        distance_travelled, current_path = heapq.heappop(paths_so_far)

    return current_path

uniform_cost()

['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']

# Sorting

Sorting algorithms are one of the most frequently studied class of algorithms. Not only are there many such algorithms, it is also very easy to implement them incorrectly.

Can we implement [Bubble sort](https://en.wikipedia.org/wiki/Bubble_sort) in Python?

In [66]:
example_list = [3,5,6,2,1]
# [3,5,2,1,6]
# [3,2,1,5,6]
# ....
# [1,2,3,5,6]

def bubble_sort(numbers):
    for _ in range(len(numbers) - 1):
        for i in range(len(numbers) - 1):
            if numbers[i] > numbers[i+1]:
                temp = numbers[i]
                numbers[i] = numbers[i+1]
                numbers[i+1] = temp

bubble_sort(example_list)
example_list

[1, 2, 3, 5, 6]

Can we also implement [merge sort](https://en.wikipedia.org/wiki/Merge_sort)?

See `sorting.py`.

# Error Handling

What do we do when things go wrong?

## Using `None`

Consider a function `find(needle, haystack)` that finds the position of `needle` in the list `haystack`. Can we implement this function?

Does python have built-in functionality for doing this?

## Exceptions

Can we change our `find()` function so that it raises an exception?

Consider the function, `score(card_points, cards)`, from the week 4 lab. It takes in a dictionary representing point values for different cards and a list of cards, and calculates the total score for that list.

Can we implement this function differently using exceptions?

A number, `x`, inside a list, `list`, is said to lead back to itself in 1 step if `list[x] == x`. A number leads back to itself in 2 steps if `list[list[x]] == x`. Which of the numbers in the following list lead back to themselves in 3 steps.

In [ ]:
list = [0, 2, 1, 4, 5, 3, 9, 6, 7]

# Alternative ways of working with data structures

Can we implement the `score(card_points, cards)` function without conditionals or exception handling?

## List/Dictionary Comprehensions

Can we extract a list of zids from these email addresses?

In [ ]:
emails = ["z1234567@student.unsw.edu.au", "z7654321@unsw.edu.au", "z7891234@ad.unsw.edu.au", "z1357924@student.unsw.edu.au"]

Can we create a dictionary with the zids as keys and the email addresses as values?

Find all the words that are palindromes in this list:

In [ ]:
words = ["kayak", "hello", "racecar", "madam", "moon", "noon", "shish", "level"]